In [ ]:
import urllib

import psycopg2
import requests

from scraper_place.config import CONFIG_DATABASE, CONFIG_ELASTICSEARCH

# Create an empty table "dce"

In [ ]:
# Open connection
connection = psycopg2.connect(
    dbname=CONFIG_DATABASE['name'],
    user=CONFIG_DATABASE['username'],
    password=CONFIG_DATABASE['password'],
)
cursor = connection.cursor()

In [ ]:
table_name = 'dce'
cursor.execute('create table dce ();')

Metadata on the DCE

In [ ]:
cursor.execute('alter table dce add column annonce_id text;')
cursor.execute('alter table dce add column org_acronym text;')
cursor.execute('alter table dce add column links_boamp text[];')
cursor.execute('alter table dce add column reference text;')
cursor.execute('alter table dce add column intitule text;')
cursor.execute('alter table dce add column objet text;')
cursor.execute('alter table dce add column reglement_ref text;')

Filenames constituting the DCE

In [ ]:
cursor.execute('alter table dce add column filename_reglement text;')
cursor.execute('alter table dce add column filename_complement text;')
cursor.execute('alter table dce add column filename_avis text;')
cursor.execute('alter table dce add column filename_dce text;')

Size of the files

In [ ]:
cursor.execute('alter table dce add column file_size_reglement bigint;')
cursor.execute('alter table dce add column file_size_complement bigint;')
cursor.execute('alter table dce add column file_size_avis bigint;')
cursor.execute('alter table dce add column file_size_dce bigint;')

Fetch date

In [ ]:
cursor.execute('alter table dce add column fetch_datetime timestamp with time zone;')

AWS Glacier archive ids

In [ ]:
cursor.execute('alter table dce add column glacier_id_reglement text;')
cursor.execute('alter table dce add column glacier_id_complement text;')
cursor.execute('alter table dce add column glacier_id_avis text;')
cursor.execute('alter table dce add column glacier_id_dce text;')

Embedded filenames for archives

In [ ]:
cursor.execute('alter table dce add column embedded_filenames_reglement text[];')
cursor.execute('alter table dce add column embedded_filenames_complement text[];')
cursor.execute('alter table dce add column embedded_filenames_avis text[];')
cursor.execute('alter table dce add column embedded_filenames_dce text[];')

The state of the processing of the DCE

In [ ]:
cursor.execute('alter table dce add column state text;')

The DCE can be queried by name and by state

In [ ]:
cursor.execute('create unique index on dce (annonce_id, org_acronym);')

In [ ]:
cursor.execute('create index on dce (state);')

In [ ]:
#cursor.execute('drop table dce;')

In [ ]:
connection.commit()

In [ ]:
cursor.close()
connection.close()

# Create the index in ElasticSearch

In [ ]:
url = urllib.parse.urljoin(
    CONFIG_ELASTICSEARCH['elasticsearch_server_url'],
    CONFIG_ELASTICSEARCH['index_name'])

In [ ]:
headers = {
    'Content-Type': 'application/json',
}
data = {
    "settings" : {
        "index" : {
            "number_of_shards" : 5, 
            "number_of_replicas" : 0,
        },
    },
}

In [ ]:
response = requests.put(url, headers=headers, json=data)
assert response.status_code == 200
response.text

In [ ]:
#response = requests.delete(url, headers=headers)
response.status_code, response.text